<h1>Write Data</h1>
I just want to write data into SQL databases for fast recall in my web application. Slow down should only occur during the scraping step and I think that is unavoidable (at the moment). The main transformation that happens in this script is taking the skills scores and the work activities scores and transforming them into a martix where each row is a job title and each column is a skill (or work activity).

In [1]:
#Import modules
import pandas as pd
import numpy
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2


#Load ONet Data
df_Skills = pd.read_excel('/Users/christopherluciuk/Desktop/Insight/JobsData/db_22_1_excel/Skills.xlsx')
df_AltTitles = pd.read_excel('/Users/christopherluciuk/Desktop/Insight/JobsData/db_22_1_excel/Alternate Titles.xlsx')
df_WorkAct = pd.read_excel('/Users/christopherluciuk/Desktop/Insight/JobsData/db_22_1_excel/Work Activities.xlsx')
df_DWARef = pd.read_excel('/Users/christopherluciuk/Desktop/Insight/JobsData/db_22_1_excel/DWA Reference.xlsx')
df_IWA = pd.read_excel('/Users/christopherluciuk/Desktop/Insight/JobsData/db_22_1_excel/DWA Reference.xlsx')

In [2]:
#Build matrix for analysis
jobsList = df_Skills['Title'].unique()
skillsList = df_Skills['Element Name'].unique()
#Make a matrix for the scores
skillsMat = numpy.zeros((len(jobsList),len(skillsList)))
for i in range(len(jobsList)):
    for j in range(len(skillsList)):
        skillsMat[i,j] = df_Skills[(df_Skills['Title']==jobsList[i]) & (df_Skills['Element Name']==skillsList[j]) & (df_Skills['Scale ID']=='IM')]['Data Value']
        
#Make a matrix of the scores        
actsList = df_WorkAct['Element Name'].unique()
actsMat = numpy.zeros((len(jobsList),len(actsList)))
outList = []
for i in range(len(jobsList)):
    for j in range(len(actsList)):
        actsMat[i,j] = df_WorkAct[(df_WorkAct['Title']==jobsList[i]) & (df_WorkAct['Element Name']==actsList[j]) & (df_WorkAct['Scale ID']=='IM')]['Data Value']       

In [7]:
#Create dataframe with desired info
df_write_skills = pd.DataFrame(skillsMat,columns=skillsList)
df_write_skills['title'] = jobsList
df_write_acts = pd.DataFrame(actsMat,columns=actsList)
df_write_acts['title'] = jobsList

#Write to database
#Create or load database
dbname = 'ONET_db_v2'
username = 'christopherluciuk' # change this to your username
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

#Write to tables
df_write_skills.to_sql('skills_table',engine,if_exists='replace')
df_write_acts.to_sql('acts_table',engine,if_exists='replace')
df_IWA.to_sql('iwa_table',engine,if_exists='replace')
df_DWARef.to_sql('dwa_table',engine,if_exists='replace')
df_AltTitles[['O*NET-SOC Code','Title','Alternate Title']].to_sql('alttitles_table',engine,if_exists='replace')

postgres://christopherluciuk@localhost/ONET_db_v2
True


,O*NET-SOC Code,Title,Alternate Title,Short Title,Source(s)
0,11-1011.00,Chief Executives,Aeronautics Commission Director,NaN,08
1,11-1011.00,Chief Executives,Agricultural Services Director,NaN,08
2,11-1011.00,Chief Executives,Alcohol and Drug Abuse Assistance Program Admi...,NaN,08
3,11-1011.00,Chief Executives,Arts and Humanities Council Director,NaN,08
4,11-1011.00,Chief Executives,Bakery Manager,NaN,08
